## Test case

I am using the herpes clade as I know a ligand exists, aciclovir,
and I know PDB exist. And it's not data explosion...

In [ ]:
herpesviridae_taxid = 10292


import pandas as pd
from boilerplate import list_tables, get_table_schema, sample, print_size
import sqlite3
import json
from pathlib import Path
from typing import List, Dict, Set

conn = sqlite3.connect('/tmp/chembl_35/chembl_35_sqlite/chembl_35.db')
cursor = conn.cursor()

In [ ]:
herpesviridae_taxid = 10292  # this seems missing for some?
herpesvirales_taxid = 548681

In [ ]:
from boilerplate import TaxIdMapper

taxid_mapper = TaxIdMapper()
taxid_to_parent = taxid_mapper.taxid_to_parent
taxid_to_parent[herpesviridae_taxid]

In [ ]:
relevant_taxids = taxid_mapper[herpesvirales_taxid]
TaxIdMapper.populate_descendants(relevant_taxids)
print(f'{len(relevant_taxids)} matches final')

In [ ]:
#varicella
10319 in relevant_taxids

In [ ]:
import json

json.dump(relevant_taxids, Path('herpesvirales_taxids.json').open('w'))

In [ ]:
import json

relevant_taxids = json.load(Path('herpesvirales_taxids.json').open('r'))

In [ ]:
#kd_like_types = ["Kd", "KD", "Potency", "Binding affinity", "pKD", "Log K", "K", "Kb", "Log Ki", "log1/Ki", "Ka"]

placeholders = ','.join(f"'{s}'" for s in relevant_taxids)
query = f"SELECT * FROM assays WHERE assay_tax_id IN ({placeholders})"
assays = pd.read_sql_query(query, conn)

In [ ]:
assays = assays.loc[assays.assay_type == 'B']
print(f'{len(assays)} biochem assays for {len(assays.assay_tax_id.unique())} taxa')

In [ ]:
from typing import Dict

taxids = assays.assay_tax_id.unique()
taxid_to_name: Dict[int, str] = {}

with open("/tmp/names.dmp") as f:
    for line in f:
        parts = [p.strip() for p in line.split("|")]
        taxid, name_txt, name_class = parts[0], parts[1], parts[3]
        if int(taxid) in taxids and name_class == "scientific name":
            taxid_to_name[int(taxid)] = name_txt

for tid in taxids:
    print(f"{tid}\t{taxid_to_name.get(tid, 'Name not found')}")

In [ ]:
placeholders = ','.join(f"'{s}'" for s in assays.assay_id)
query = f"SELECT * FROM activities WHERE assay_id IN ({placeholders})"
activities = pd.read_sql_query(query, conn)

In [ ]:
activities['unit_scale']=activities.standard_units.apply(molar_scale)

In [ ]:
activities['assay_tax_id'] = activities.assay_id.map( assays.set_index('assay_id').assay_tax_id.to_dict() ).fillna(-1)
activities['assay_organism'] = activities.assay_tax_id.map(taxid_to_name)

In [ ]:
import plotly.express as px
import numpy as np

activities['pK'] = - (activities.unit_scale * activities.standard_value).apply(np.log10)

sub = activities.loc[(~activities.pK.isna())].sort_values('pK', ascending=False).drop_duplicates(['molregno', 'assay_organism'])
px.box(sub, x='pK',
             y='assay_organism',
             template='plotly_white',
             title=f'Distribution of {len(sub):,} pK activities (Kd, Ki, IC50 etc not separated) of herpesvirales',).show()

sub = activities.loc[(activities.standard_type == 'Kd') & (~activities.pK.isna())].sort_values('pK', ascending=False).drop_duplicates(['molregno', 'assay_organism'])
px.box(sub,
       points='all',
       x='pK',
             y='assay_organism',
             template='plotly_white',
             title=f'Distribution of {len(sub)} pKd activities of herpesvirales',).show()


In [ ]:
placeholders = ', '.join(activities.molregno.astype(str).apply("'{}'".format).unique())

query = f"SELECT * FROM COMPOUND_STRUCTURES WHERE molregno IN ({placeholders})"
molregno = pd.read_sql_query(query, conn)

In [ ]:
activities['standard_inchi_key'] = activities.molregno.map( molregno.set_index('molregno').standard_inchi_key.to_dict() )

## Get PDB using Papyrus

This is a problematic.

In [ ]:
PDB_matches = pd.read_csv('PDB_matches.csv')  # papyrus. crap it's chembl34
PDB_matches.columns

A compound may be tested in multiple assays,
so this first pass is 'imperfect'

In [ ]:
imperfect_pdb = pd.merge(activities, PDB_matches, how='inner', left_on='standard_inchi_key', right_on='InChIKey')

In [ ]:
present_assay_ids = set( assays.assay_id.values )

all_matches = []
for aid in imperfect_pdb.AID:
    matches = set(map(int, aid.replace('CHEMBL', '').split(';'))).intersection( present_assay_ids )
    if matches:
        print(matches)
        print(aid)
        all_matches.append(matches)

print(f'Number matched {len(all_matches)}')

The above is not good...

## Aciclovir test

In [ ]:
molregno.loc[molregno.standard_inchi_key == 'MKUXAQIIEYXACX-UHFFFAOYSA-N']

In [ ]:
activities.loc[activities.molregno == 365538].groupby('assay_organism').standard_type.value_counts()

In [ ]:
activities.loc[activities.molregno == 365538].assay_id.unique()

In [ ]:
PDB_matches.loc[PDB_matches.InChIKey == 'MKUXAQIIEYXACX-UHFFFAOYSA-N']\
[['PDBID_protein', 'AID', 'target_id', 'TID', 'accession', 'uniprot_tax_id']]

In [ ]:
print(_.to_markdown(index=False))

In [ ]:
assays.tid

In [ ]:
imperfect_pdb.loc[imperfect_pdb.InChIKey == 'MKUXAQIIEYXACX-UHFFFAOYSA-N']\
[['activity_id','assay_id','record_id', 'PDBID_protein', 'AID', 'target_id', 'TID', 'accession']]

I am struggling to match Chembl assay ID to the papyrus data.
Plus the Chembl34 is a problem.

Trying the new https://www.ebi.ac.uk/about/news/updates-from-data-resources/biochemgraph-data

## biochemgraph

This set goes by `CHEMBL_ID_TAR` not assay.

In [ ]:
import gzip
from typing import Dict

def load_uniprot_taxid_mapping(mapping_file: str) -> Dict[str, int]:
    mapping = {}
    with gzip.open(mapping_file, 'rt') as f:
        for line in f:
            parts = line.strip().split('\t')
            mapping[parts[0].strip()] = int(parts[12].strip())
            mapping[parts[1].strip()] = int(parts[12].strip())
    return mapping

uniprot2taxid = load_uniprot_taxid_mapping('idmapping_selected.tab.gz')

In [ ]:
import json

json.dump(uniprot2taxid, Path('uniprot2taxid.json').open('w'))

In [ ]:
import pandas as pd

bioactivities = pd.read_csv('bioactivity_report_full.tsv', delimiter='\t')

In [ ]:
len(bioactivities)

In [ ]:
from tqdm import tqdm
tqdm.pandas()
import gzip
from typing import Dict

_dejavu={}
def get_taxid(uniprot, mapping_file: str='idmapping_selected.tab.gz') -> int:
    if uniprot in _dejavu:
        return _dejavu[uniprot]
    with gzip.open(mapping_file, 'rt') as f:
        for line in f:
            if uniprot in line:
                parts = line.strip().split('\t')
                _dejavu[uniprot] = int(parts[12].strip())
                return int(parts[12].strip())
    return -1

bioactivities['tax_id'] = bioactivities.UNIPROT.progress_apply(get_taxid)

In [ ]:
bioactivities.to_pickle('bioactivities.pkl.gz')

In [ ]:
#bioactivities # CHEMBL_ID_TAR CHEMBL_ID_MOL STANDARD_TYPE
len(bioactivities)

In [ ]:
bioactivities['tax_id']

In [ ]:
from boilerplate import TaxIdMapper

In [ ]:
# is a human a mammal?
taxidmapper = TaxIdMapper(target_ids=(40674,))
taxidmapper.has_ancestor_in_set(9606)

In [ ]:
herpesviridae_taxid = 10292  # this seems missing for some?
herpesvirales_taxid = 548681

bioactivities['is_herpesvirales'] = bioactivities.tax_id.progress_apply(TaxIdMapper(target_ids=(herpesvirales_taxid,herpesviridae_taxid)).has_ancestor_in_set)

In [ ]:
bioactivities.loc[bioactivities['is_herpesvirales']]

In [ ]:
PDB_matches['uniprot_tax_id'] = PDB_matches.accession.progress_apply(get_taxid)

In [ ]:
target_ids={herpesvirales_taxid,herpesviridae_taxid}
PDB_matches['is_herpesvirales'] = PDB_matches.uniprot_tax_id.progress_apply(taxid_mapper.has_ancestor_in_set)

In [ ]:
PDB_matches.loc[PDB_matches['is_herpesvirales']]

## PDB

In [ ]:
PDB_matches.loc[PDB_matches.PDBID_protein.isin(list(map(str.lower, '8V1T 7LUF 8VQ2 1NKM 8J3T 7TCZ'.split())))]

## TID

In [ ]:
see in junk

## Save

In [ ]:
activities.to_csv('herpesvirales_activities.csv')

In [ ]:
bioactivities.drop(columns=['is_herpesvirales']).to_csv('bioactivities_w_taxid.csv')

# Junk

In [ ]:
activities.loc[activities.assay_tax_id.isna()]

In [ ]:
assays.loc[~assays.assay_id.isin(assays.assay_id.values)]

In [ ]:
activities.loc[activities.assay_tax_id.isna()]

In [ ]:
activities.assay_organism.fillna('NaN').value_counts()

In [ ]:
activities.assay_id.fillna(-1).value_counts()

In [ ]:
assays.columns

In [ ]:
assays[['assay_id', 'tid', 'description', 'assay_tax_id']].sample(5)

In [ ]:
assays.tid.fillna('NA').value_counts().head(10)

In [ ]:
print(_.to_markdown(index=False))

In [ ]:
query = f"SELECT * FROM target_components WHERE tid IN (22226,100288,10125)"
target_components = pd.read_sql_query(query, conn)
target_components

In [ ]:
print(_.to_markdown(index=False))

In [ ]:

query = f"SELECT * FROM target_dictionary WHERE tid IN (98,22222,10125,22226,100288,10125)"
target_dictionary = pd.read_sql_query(query, conn)
target_dictionary

In [ ]:
print(_.to_markdown(index=False))

In [ ]:
JOIN target_dictionary td ON a.tid = td.target_id
JOIN target_components tc ON td.target_id = tc.target_id
JOIN component_sequences cs ON tc.component_id = cs.component_id

In [ ]:
sample(conn, 'target_dictionary')

In [ ]:
sample(conn, 'component_sequences')

In [ ]:
len(taxid2rank)